In [1]:
import openpyxl
from pyproj import Transformer
import json

# Crear un transformador de coordenadas de UTM (Zona 18S) a WGS84 (latitud/longitud)
transformer = Transformer.from_crs("EPSG:32718", "EPSG:4326", always_xy=True)

# Función para convertir coordenadas UTM a latitud y longitud
def utm_to_latlon(east, north):
    lon, lat = transformer.transform(east, north)
    return lat, lon

# Leer el archivo Excel y convertir los datos
def process_water_bodies(file_path):
    water_bodies = []
    # Cargar el archivo Excel
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active  # Usar la hoja activa

    # Obtener los encabezados
    headers = [cell.value for cell in sheet[1]]

    # Iterar sobre las filas de datos
    for row in sheet.iter_rows(min_row=2, values_only=True):
        row_data = dict(zip(headers, row))
        
        # Convertir valores a float y manejar excepciones
        try:
            east = float(row_data['ESTE'])
            north = float(row_data['NORTE'])
        except (ValueError, TypeError) as e:
            print(f"Error en las coordenadas UTM: {e}")
            continue
        
        lat, lon = utm_to_latlon(east, north)
        
        water_bodies.append({
            "name": row_data['NOMBRE DEL RECURSO HÍDRICO'],
            "description": row_data['DESCRIPCION'],
            "coordinates": [lat, lon],
            "altitude": int(row_data['ALTITUD m.s.n.m.']),
            "category": row_data['CLASIFICACÍON DE CUERPOS DE AGUA']
        })
    return water_bodies

# Guardar los datos en formato JSON
def save_to_json(data, output_path):
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

# Archivo de entrada
file_path = "../dataset/RedMonitoreo6102024115440.xlsx"
# Archivo de salida
output_json_path = "../dataset/agua_cuerpos.json"

# Convertir los puntos de agua y guardarlos en JSON
water_bodies = process_water_bodies(file_path)
save_to_json(water_bodies, output_json_path)

# Mostrar los primeros 5 cuerpos de agua convertidos
for water_body in water_bodies[:5]:
    print(water_body)


{'name': 'VILCANOTA', 'description': 'Río Vilcanota, aguas abajo km 107 Machupicchu', 'coordinates': [-13.177676167759769, -72.53150439410855], 'altitude': 2167, 'category': 'Categoría 4'}
{'name': 'Vilcanota', 'description': 'Río Vilcanota, km 104 de la línea ferrea de Ollantaytambo a Machupicchu.\n', 'coordinates': [-13.185673522211403, -72.51110174671373], 'altitude': 2193, 'category': 'Categoría 4'}
{'name': 'Vilcanota', 'description': 'Río Vilcanota; aproximadamente a 2 km aguas arriba del centro poblado Machupicchu Pueblo.\n', 'coordinates': [-13.160995322058966, -72.5228390092081], 'altitude': 2092, 'category': 'Categoría 4'}
{'name': 'Vilcanota', 'description': 'Rio Vilcanota; aproximadamente a 400 m aguas abajo del centro poblado de Machupicchu Pueblo.', 'coordinates': [-13.156418960468134, -72.53079544387526], 'altitude': 2083, 'category': 'Categoría 4'}
{'name': 'VILCANOTA', 'description': 'Río Vilcanota 350 m aprox. Aguas abajo del vertimiento de aguas residuales de Santuar

In [3]:
water_body

{'name': 'VILCANOTA',
 'description': 'Río Vilcanota 350 m aprox. Aguas abajo del vertimiento de aguas residuales de Santuary Lodge- margen derecha siguiendo las rieles del tren',
 'coordinates': [-13.15822038714648, -72.54150919624414],
 'altitude': 1993,
 'category': 'Categoría 4'}